In [1]:
import tomica
print(tomica.__file__)

d:\tomica\tomica\__init__.py


In [17]:
import polars as pl
import random
import time

In [3]:
data = pl.select(
    pl.lit(1).sample(n=1e6, with_replacement=True)  # 先采样
    .map_elements(lambda _: random.uniform(-50, 50), return_dtype=pl.Float64)
).to_series()

In [24]:
def test_speed(fn,window_size):
    st= time.time()
    fn(data,window_size)
    print(f'{fn.__name__}: {time.time()-st:.4f}s')

In [27]:
for func in tomica.operators.__all__:
    func = getattr(tomica.operators,func)
    test_speed(func,10)

ts_delay: 0.0011s
ts_delta: 0.0600s
ts_returns: 0.0126s
ts_sum: 0.0186s
ts_product: 0.0160s
ts_max: 0.0305s
ts_min: 0.0237s
ts_mean: 0.0141s
ts_std: 0.0205s
ts_rank: 0.2281s
ts_variance: 0.0169s
ts_quantile_up: 0.0675s
ts_quantile_down: 0.0671s


In [6]:
tomica.operators.ts_delta(data,1)

literal
f64
null
61.159617
14.833369
-26.473557
-33.219641
…
21.179969
3.704786
-13.510789


In [7]:
tomica.operators.ts_returns(data,1)

literal
f64
null
-1.54671
0.686162
-0.726273
-3.329391
…
1.950614
0.115637
-0.378


In [8]:
tomica.operators.ts_sum(data,1)

literal
f64
-39.541738
21.617879
36.451248
9.977692
-23.241949
…
32.038074
35.74286
22.232071


In [9]:
tomica.operators.ts_mean(data,3)

literal
f64
-39.541738
-8.961929
6.175797
22.682273
7.728997
…
3.320476
26.213013
30.004335


In [10]:
tomica.operators.ts_max(data,3)

literal
f64
-39.541738
21.617879
36.451248
36.451248
36.451248
…
32.038074
35.74286
35.74286


In [11]:
tomica.operators.ts_min(data,3)

literal
f64
-39.541738
-39.541738
-39.541738
9.977692
-23.241949
…
-32.934751
10.858105
22.232071


In [12]:
tomica.operators.ts_std(data,3)

literal
f64
null
43.24638
40.281223
13.268836
29.910064
…
33.135764
13.426141
6.981219


In [13]:
tomica.operators.ts_rank(data,3)

literal
f64
1.0
2.0
3.0
1.0
1.0
…
3.0
3.0
1.0


In [14]:
tomica.operators.ts_variance(data,3)

literal
f64
null
1870.249371
1622.576917
176.062
894.611933
…
1097.978883
180.261264
48.737423


In [15]:
tomica.operators.ts_quantile_up(data,10)

literal
f64
-39.541738
21.617879
36.451248
21.617879
21.617879
…
32.038074
35.228491
35.228491


In [16]:
tomica.operators.ts_quantile_down(data,10)

literal
f64
-39.541738
-39.541738
21.617879
9.977692
-23.241949
…
-26.737458
-26.737458
-26.737458
